In [2]:

import pandas as pd
def organizar_lat_long(latitude, longitude):
    # Verificar se os valores são "sem informações" e retorná-los sem alterações
    if latitude.lower() == "sem informações" or longitude.lower() == "sem informações":
        return latitude, longitude

    # Remover todos os pontos das latitudes e longitudes
    latitude = latitude.replace('.', '')
    longitude = longitude.replace('.', '')

    if not latitude.startswith('-'):
        latitude = '-' + latitude
    if not longitude.startswith('-'):
        longitude = '-' + longitude
    # Adicionar o '.' após o terceiro caracter, se houver pelo menos três caracteres
    if len(latitude) >= 3:
        latitude = latitude[:3] + '.' + latitude[3:]
    if len(longitude) >= 3:
        longitude = longitude[:3] + '.' + longitude[3:]

    
    if -39.00 <= float(latitude) <= -37.00:
        return longitude, latitude  # Inverter latitude e longitude
    else:
        return latitude, longitude

    return latitude, longitude

# Ler o arquivo CSV
df = pd.read_csv('RegistroSE.csv', sep=',')

# Aplicar a função de organização de latitude e longitude para cada linha do DataFrame
df['Latitude'], df['Longitude'] = zip(*df.apply(lambda row: organizar_lat_long(row['Latitude'], row['Longitude']), axis=1))

# Remover espaços em branco das colunas Latitude e Longitude, exceto para a opção "sem informações"
df.loc[df['Latitude'].str.lower() != 'sem informações', 'Latitude'] = df['Latitude'].str.strip()
df.loc[df['Longitude'].str.lower() != 'sem informações', 'Longitude'] = df['Longitude'].str.strip()

# Salvar o DataFrame de volta para um novo arquivo CSV
df.to_csv('registroSE_organizado.csv', sep=',', index=False)

In [4]:
import folium
import pandas as pd

def convert_to_float(value):
    if value == 'Sem Informações':
        return value
    return float(value)

# Carregue os dados do arquivo CSV
a = pd.read_csv('./registroSE_organizado.csv', sep=',')

# Filtrar registros com datas após 2012
a['Data do registro'] = pd.to_datetime(a['Data do registro'], format='%d/%m/%Y')
a = a[a['Data do registro'].dt.year >= 2012]

# Converter as colunas de Latitude e Longitude para float, exceto "Sem Informações"
a['Latitude'] = a['Latitude'].apply(convert_to_float)
a['Longitude'] = a['Longitude'].apply(convert_to_float)

# Filtrar "Sem Informações"
a_filtered = a[(a['Latitude'] != 'Sem Informações') & (a['Longitude'] != 'Sem Informações')]

# Criar o mapa
mapa = folium.Map(location=[a_filtered['Latitude'].mean(), a_filtered['Longitude'].mean()], zoom_start=6)

# Contagem de "Sem Informações"
sem_informacoes = a[(a['Latitude'] == 'Sem Informações') | (a['Longitude'] == 'Sem Informações')].shape[0]

# Adicionar círculo cinza com popup da contagem de "Sem Informações"
folium.Marker(
    location=[-10.9092, -37.0674],
    popup=f"Aracaju-SE: <br> Pontos Sem Informações: {sem_informacoes}",
    icon=folium.Icon(color='blue') 
).add_to(mapa)
# Adicionar pequenos círculos para cada endereço com borda preta
for index, row in a_filtered.iterrows():
    # Cria um círculo pequeno para cada endereço
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,  # Defina o raio do círculo pequeno (em pixels)
        color='green',  # Cor do círculo pequeno
        fill=True,
        fill_color='green',
        fill_opacity=0.4,
        weight=1,
        popup=f"Nome Comum: {row['Nome comum']}<br>Latitude: {row['Latitude']}<br>Longitude: {row['Longitude']}"
    ).add_to(mapa)

# Exibir o mapa interativo
mapa
